# <u> Hybrid Content-Collaborative Filtering Model</u>


## <u>0. Setting:</u>

### <u>0.1 Import libraries</u>

In [ ]:
# Import necessary libraries
import pandas as pd, numpy as np, os, sys
import time
import matplotlib.pyplot as plt
from lightfm import LightFM

# Remove userwarnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


# Set the working directory
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
# Import necessary libraries
import pandas as pd, numpy as np, os, sys
import time
import matplotlib.pyplot as plt
from lightfm import LightFM

# Remove userwarnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


# Set the working directory
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

### <u>0.2 Import pre-built datasets</u>

For consistency with the previous experiments, we adopt the same **time-based train–validation–test split**. For each user, the **earliest 70% of ratings** are used for training, the **next 10% for validation**, and the **most recent 20% for testing**. This ensures a fair comparison across models while preventing information leakage into the evaluation set.

Given the hybrid nature of the hybrid algorithm, which jointly leverages **collaborative interactions and content-based features**, the movie feature vectors constructed in `05_content_F.ipynb` are used as item-information. In addition, user-specific feature representations are constructed and incorporated into the model. This allows the model to learn latent representations that combine interaction patterns with explicit content information in opposite to its baseline model evalauated in `04_model_CF.ipynb`.

Following the evaluation protocol used for collaborative and model-based approaches, RMSE is reported on the full test set as well as separately for **warm-start** and **cold-start** settings. The cold-start subset corresponds to movies with fewer than 10 interactions in the training set, where predictions rely more heavily on content features rather than collaborative signals. Additionally, Hyperparameter tuning is performed on the validation set, and the total training and evaluation time is reported to enable computational performance comparisons.


In [ ]:
#Load dataframe over the columns of interest
train_df = pd.read_csv('../data/processed/train_df.csv')
val_df = pd.read_csv('../data/processed/val_df.csv')
test_df = pd.read_csv('../data/processed/test_df.csv')
warm_test_df = pd.read_csv('../data/processed/warm_test_df.csv')
cold_test_df = pd.read_csv('../data/processed/cold_test_df.csv')
movies_vector_std = pd.read_csv('../data/processed/movies_vector_std.csv')